# Neural Network Classifiers

In [2]:
import pandas as pd

# Import data
train = pd.read_csv("../data/train.csv").sort_values('id')
train_df = train.loc[:, train.columns != 'price']
train_prices = train['price'].values

test_df = pd.read_csv("../data/test.csv").sort_values('id')

In [4]:
from data_processing import preprocessing

# Combine data for processing (important for one-hot encoding)
combined_df = pd.concat([test_df, train_df], ignore_index=True)

# Select which columns to include in the analysis
selected_features = list(combined_df.columns)
to_remove = ['id', 'scrape_id', 'last_scraped', 'name', 'description', 
             'picture_url', 'host_id', 'host_name', 'calendar_last_scraped',
             'bathrooms_text']
for col in to_remove:
  selected_features.remove(col)

# Process combined data
processed_data = preprocessing(combined_df.copy(), selected_features)

test_processed = processed_data.iloc[:len(test_df)]
train_processed = processed_data.iloc[len(test_df):]

### Limited features testing

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Select 7 prominent features to train on
selected_features = ['latitude', 'longitude', 'accommodates', 
                     'amenities', 'availability_365', 
                     'number_of_reviews_ltm', 'host_since']

X_train = train_processed[selected_features].copy()
y_train = tf.keras.utils.to_categorical(train_prices, num_classes=6)

# Build the neural network model
model = Sequential()
model.add(Dense(10, input_dim=7, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(6, activation='softmax'))  # Output layer with softmax activation for classification

# Compile the model
model.compile(loss='categorical_crossentropy',  # Use categorical crossentropy for multi-class classification
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

2023-12-05 18:45:16.298378: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10


2023-12-05 18:45:37.923716: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


483/483 [==============================] - 1s 1ms/step - loss: 1.7438 - accuracy: 0.2532
Epoch 2/10
483/483 [==============================] - 1s 1ms/step - loss: 1.7349 - accuracy: 0.2598
Epoch 3/10
483/483 [==============================] - 1s 1ms/step - loss: 1.7348 - accuracy: 0.2598
Epoch 4/10
483/483 [==============================] - 1s 1ms/step - loss: 1.7335 - accuracy: 0.2598
Epoch 5/10
483/483 [==============================] - 1s 1ms/step - loss: 1.7141 - accuracy: 0.2598
Epoch 6/10
483/483 [==============================] - 1s 1ms/step - loss: 1.5739 - accuracy: 0.3367
Epoch 7/10
483/483 [==============================] - 1s 2ms/step - loss: 1.4950 - accuracy: 0.3775
Epoch 8/10
483/483 [==============================] - 1s 1ms/step - loss: 1.4753 - accuracy: 0.3805
Epoch 9/10
483/483 [==============================] - 1s 1ms/step - loss: 1.4635 - accuracy: 0.3927
Epoch 10/10
483/483 [==============================] - 1s 1ms/step - loss: 1.4533 - accuracy: 0.4055


### Larger model

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Select 7 prominent features to train on
selected_features = ['latitude', 'longitude', 'accommodates', 
                     'amenities', 'availability_365', 
                     'number_of_reviews_ltm', 'host_since', 'host_listings_count',
                     'host_is_superhost', 'host_identity_verified', 
                     'neighbourhood_cleansed', 'room_type', 'has_availability',
                     'availability_60', 'calculated_host_listings_count']
X_train = train_processed[selected_features].copy()
# Convert output data to one-hot encoding
y_train = tf.keras.utils.to_categorical(train_prices, num_classes=6)

# Build the neural network model
model = Sequential()
model.add(Dense(32, input_dim=15, activation='sigmoid'))
model.add(Dense(16, activation='sigmoid'))
model.add(Dense(12, activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(6, activation='softmax'))  # Output layer with softmax activation for classification

# Compile the model
model.compile(loss='categorical_crossentropy',  # Use categorical crossentropy for multi-class classification
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=15, batch_size=32)

Epoch 1/15
483/483 [==============================] - 1s 2ms/step - loss: 1.7557 - accuracy: 0.2431
Epoch 2/15
483/483 [==============================] - 1s 2ms/step - loss: 1.7347 - accuracy: 0.2598
Epoch 3/15
483/483 [==============================] - 1s 2ms/step - loss: 1.7280 - accuracy: 0.2598
Epoch 4/15
483/483 [==============================] - 1s 1ms/step - loss: 1.6656 - accuracy: 0.2962
Epoch 5/15
483/483 [==============================] - 1s 1ms/step - loss: 1.5571 - accuracy: 0.3684
Epoch 6/15
483/483 [==============================] - 1s 1ms/step - loss: 1.5083 - accuracy: 0.3853
Epoch 7/15
483/483 [==============================] - 1s 2ms/step - loss: 1.4833 - accuracy: 0.3910
Epoch 8/15
483/483 [==============================] - 1s 1ms/step - loss: 1.4651 - accuracy: 0.3880
Epoch 9/15
483/483 [==============================] - 1s 1ms/step - loss: 1.4532 - accuracy: 0.3945
Epoch 10/15
483/483 [==============================] - 1s 1ms/step - loss: 1.4420 - accuracy: 0.4031

### Attempts to include images and descriptions

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, Embedding, Flatten
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import requests
from io import BytesIO
from PIL import Image
import tqdm

# Function to load and preprocess images
def load_and_preprocess_images(ids, max_samples=None):
    image_data = []
    inclusion_mask = []

    for id in tqdm.tqdm(ids[:max_samples], desc="Loading Images", unit="image"):
        try:
            img = Image.open(f'../data/test_images/{id}.jpg')
            img = img.resize((224, 224))  # Resize images to match MobileNetV2 input size
            img_array = image.img_to_array(img)
            img_array = preprocess_input(img_array)

            # Ensure the image array has the correct shape
            if img_array.shape == (224, 224, 3):
                image_data.append(img_array)
                inclusion_mask.append(True)
            else:
                inclusion_mask.append(False)
        except Exception as e:
            inclusion_mask.append(False)

    return np.array(image_data), inclusion_mask

max_samples = 1000
image_data, inclusion_mask = load_and_preprocess_images(train_df['id'], max_samples)

Loading Images: 100%|██████████| 1000/1000 [02:36<00:00,  6.38image/s]


In [16]:
# Function to tokenize and pad sequences for descriptions
def preprocess_descriptions(descriptions, max_samples=None):
    descriptions = [str(desc) for desc in descriptions[:max_samples]]
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(descriptions[:max_samples])
    sequences = tokenizer.texts_to_sequences(descriptions[:max_samples])
    padded_sequences = pad_sequences(sequences)
    return padded_sequences, tokenizer

description_data, tokenizer = preprocess_descriptions(train_df['description'][:max_samples][inclusion_mask])
price_data = np.array(train_prices[:max_samples][inclusion_mask])

In [17]:
# Split the data into training and testing sets
image_train, image_test, desc_train, desc_test, price_train, price_test = train_test_split(
    image_data, description_data, price_data, test_size=0.2, random_state=42
)

# Load pre-trained MobileNetV2 model for image processing
image_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained image model
for layer in image_model.layers:
    layer.trainable = False

# Create the text processing branch
desc_input = Input(shape=(description_data.shape[1],))
desc_embedding = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=description_data.shape[1])(desc_input)
desc_flatten = Flatten()(desc_embedding)

# Flatten the output of the MobileNetV2 layer
image_flatten = Flatten()(image_model.output)

# Combine the flattened image and text features
combined = concatenate([image_flatten, desc_flatten])

# Add additional layers for the final prediction
x = Dense(128, activation='relu')(combined)
output = Dense(1, activation='linear')(x)

# Create the final model
model = Model(inputs=[image_model.input, desc_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'accuracy'])

# Train the model
model.fit([np.asarray(image_train), np.asarray(desc_train)], np.asarray(price_train), epochs=10, batch_size=32, validation_data=([np.asarray(image_test), np.asarray(desc_test)], np.asarray(price_test)))

9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/10
24/24 [==============================] - 22s 816ms/step - loss: 648.3593 - mae: 10.8530 - accuracy: 0.2497 - val_loss: 6.5833 - val_mae: 1.9903 - val_accuracy: 0.2474
Epoch 2/10
24/24 [==============================] - 17s 729ms/step - loss: 7.7163 - mae: 2.0483 - accuracy: 0.2510 - val_loss: 6.5402 - val_mae: 1.9826 - val_accuracy: 0.2474
Epoch 3/10
24/24 [==============================] - 19s 818ms/step - loss: 6.5868 - mae: 1.9866 - accuracy: 0.2510 - val_loss: 6.5277 - val_mae: 1.9810 - val_accuracy: 0.2474
Epoch 4/10
24/24 [==============================] - 19s 772ms/step - loss: 6.5539 - mae: 1.9818 - accuracy: 0.2510 - val_loss: 6.5134 - val_mae: 1.9792 - val_accuracy: 0.2474
Epoch 5/10
24/24 [==============================] - 18s 757ms/step - loss: 6.5292 - mae: 1.9786 - accuracy: 0.2510 - val_loss: 6.4971 - val_mae: 1.9771 - val_accuracy: 0.2474
Epoch 6/10
24/24 [==============================] - 18s 776